In [1]:
import requests
import pandas as pd
from datetime import datetime
import schedule
import time

In [2]:
# Initialize a DataFrame to store weather data
weather_df = pd.DataFrame(columns=['timestamp', 'city', 'temperature', 'humidity', 'pressure', 'weather', 'wind_speed', 'season'])

In [3]:
# Your OpenWeatherMap API key
api_key = '429e6570bf176b57734d33f03e17ea4a'

In [4]:
# List of cities you want to fetch weather data 
cities = ['North and Middle Andaman ', 'Chittor', 'Guntur', 'Visakhapatnam',
       'Barpeta', 'Cachar', 'Goalpara', 'Jorhat', 'Kamrup',
       'Karbi Anglong', 'Kokrajhar', 'MORIGAON', 'Sonitpur', 'Araria',
       'Bhojpur', 'Madhubani', 'Rohtas', 'Chandigarh', 'Bilaspur',
       'Dhamtari', 'Durg', 'Kondagaon', 'Raipur', 'Rajnandgaon',
       'North Goa', 'South Goa', 'Ahmedabad', 'Amreli', 'Anand',
       'Banaskanth', 'Bharuch', 'Bhavnagar', 'Chhota Udaipur', 'Dahod',
       'Dang', 'Devbhumi Dwarka', 'Gandhinagar', 'Gir Somnath',
       'Jamnagar', 'Junagarh', 'Kachchh', 'Kheda', 'Mehsana', 'Morbi',
       'Navsari', 'Panchmahals', 'Patan', 'Porbandar', 'Rajkot',
       'Sabarkantha', 'Surat', 'Surendranagar', 'Vadodara(Baroda)',
       'Valsad', 'Ambala', 'Bhiwani', 'Fatehabad', 'Gurgaon', 'Hissar',
       'Jhajar', 'Jind', 'Kaithal', 'Kurukshetra', 'Mahendragarh-Narnaul',
       'Mewat', 'Panchkula', 'Panipat', 'Rewari', 'Rohtak', 'Sonipat',
       'Yamuna Nagar', 'Chamba', 'Hamirpur', 'Kangra', 'Kullu', 'Mandi',
       'Shimla', 'Una', 'Anantnag', 'Badgam', 'Baramulla', 'Jammu',
       'Kathua', 'Pulwama', 'Rajouri', 'Srinagar', 'Udhampur',
       'Bangalore', 'Chamrajnagar', 'Chikmagalur', 'Chitradurga',
       'Davangere', 'Kolar', 'Koppal', 'Mandya',
       'Mangalore(Dakshin Kannad)', 'Mysore', 'Raichur', 'Shimoga',
       'Udupi', 'Alappuzha', 'Ernakulam', 'Idukki', 'Kannur', 'Kasargod',
       'Kollam', 'Kottayam', 'Kozhikode(Calicut)', 'Malappuram',
       'Palakad', 'Pathanamthitta', 'Thirssur', 'Thiruvananthapuram',
       'Wayanad', 'Alirajpur', 'Anupur', 'Ashoknagar', 'Badwani',
       'Balaghat', 'Bhind', 'Bhopal', 'Chhatarpur', 'Chhindwara', 'Datia',
       'Dewas', 'Dhar', 'Dindori', 'Guna', 'Gwalior', 'Harda',
       'Hoshangabad', 'Indore', 'Khandwa', 'Khargone', 'Mandla',
       'Mandsaur', 'Morena', 'Narsinghpur', 'Neemuch', 'Rajgarh',
       'Ratlam', 'Sagar', 'Satna', 'Sehore', 'Seoni', 'Shajapur',
       'Sheopur', 'Tikamgarh', 'Ujjain', 'Vidisha', 'Ahmednagar', 'Akola',
       'Buldhana', 'Kolhapur', 'Mumbai', 'Nagpur', 'Nashik', 'Pune',
       'Raigad', 'Ratnagiri', 'Sangli', 'Satara', 'Sholapur', 'Thane',
       'Yavatmal', 'East Khasi Hills', 'Kohima', 'Mokokchung', 'Delhi',
       'Balasore', 'Bargarh', 'Boudh', 'Dhenkanal', 'Kalahandi',
       'Kendrapara', 'Keonjhar', 'Nayagarh', 'Nuapada', 'Rayagada',
       'Sundergarh', 'Amritsar', 'Bhatinda', 'Faridkot', 'Fatehgarh',
       'Fazilka', 'Ferozpur', 'Gurdaspur', 'Hoshiarpur', 'Jalandhar',
       'Ludhiana', 'Mansa', 'Moga', 'Mohali', 'Nawanshahr', 'Pathankot',
       'Patiala', 'Ropar (Rupnagar)', 'Sangrur', 'Tarntaran', 'Ajmer',
       'Baran', 'Bikaner', 'Bundi', 'Churu', 'Ganganagar', 'Hanumangarh',
       'Jalore', 'Jodhpur', 'Jodhpur Rural', 'Pratapgarh', 'Sikar',
       'Tonk', 'Udaipur', 'Ariyalur', 'Chengalpattu', 'Coimbatore',
       'Cuddalore', 'Dharmapuri', 'Dindigul', 'Erode', 'Kallakuruchi',
       'Kancheepuram', 'Karur', 'Krishnagiri', 'Madurai', 'Nagapattinam',
       'Nagercoil (Kannyiakumari)', 'Namakkal', 'Perambalur',
       'Pudukkottai', 'Ramanathapuram', 'Ranipet', 'Salem', 'Sivaganga',
       'Tenkasi', 'Thanjavur', 'The Nilgiris', 'Theni',
       'Thiruchirappalli', 'Thirunelveli', 'Thirupathur', 'Thirupur',
       'Thiruvannamalai', 'Thiruvarur', 'Thiruvellore', 'Tuticorin',
       'Vellore', 'Villupuram', 'Virudhunagar', 'Hyderabad', 'Karimnagar',
       'Khammam', 'Mahbubnagar', 'Medak', 'Nalgonda', 'Nizamabad',
       'Ranga Reddy', 'Warangal', 'Gomati', 'North Tripura', 'Sepahijala',
       'Unokoti', 'Agra', 'Aligarh', 'Ambedkarnagar', 'Amethi', 'Amroha',
       'Auraiya', 'Ayodhya', 'Azamgarh', 'Badaun', 'Baghpat', 'Bahraich',
       'Ballia', 'Balrampur', 'Banda', 'Barabanki', 'Basti',
       'Bhadohi(Sant Ravi Nagar)', 'Bijnor', 'Bulandshahar', 'Chandauli',
       'Etah', 'Etawah', 'Farukhabad', 'Fatehpur', 'Firozabad',
       'Gautam Budh Nagar', 'Ghaziabad', 'Ghazipur', 'Gonda', 'Gorakhpur',
       'Hardoi', 'Jalaun (Orai)', 'Jaunpur', 'Jhansi', 'Kannuj', 'Kanpur',
       'Kasganj', 'Kaushambi', 'Khiri (Lakhimpur)', 'Lakhimpur',
       'Lucknow', 'Maharajganj', 'Mahoba', 'Mainpuri', 'Mathura',
       'Mau(Maunathbhanjan)', 'Meerut', 'Mirzapur', 'Muzaffarnagar',
       'Pillibhit', 'Prayagraj', 'Raebarelli', 'Rampur', 'Saharanpur',
       'Sambhal', 'Sant Kabir Nagar', 'Shahjahanpur', 'Shamli',
       'Siddharth Nagar', 'Sitapur', 'Unnao', 'Varanasi', 'Champawat',
       'Dehradoon', 'Haridwar', 'Nanital', 'UdhamSinghNagar',
       'Alipurduar', 'Bankura', 'Birbhum', 'Darjeeling', 'Howrah',
       'Jalpaiguri', 'Kolkata', 'Medinipur(E)', 'Medinipur(W)',
       'Murshidabad', 'Nadia', 'North 24 Parganas', 'Paschim Bardhaman',
       'Purba Bardhaman', 'Puruliya', 'Sounth 24 Parganas',
       'East Godavari', 'Sibsagar', 'Vaishali', 'Surguja', 'Dharwad',
       'Kalburgi', 'Yadgiri', 'Damoh', 'Jabalpur', 'Katni', 'Panna',
       'Raisen', 'Shehdol', 'Umariya', 'Chandrapur', 'Hingoli', 'Latur',
       'Nanded', 'West Garo Hills', 'Dimapur', 'Bhadrak', 'Balotra',
       'Barmer', 'Dausa', 'Dungarpur', 'Nagaur', 'Neem Ka Thana',
       'Bareilly', 'Hathras', 'Lalitpur', 'Uttar Dinajpur', 'Nellore',
       'Darbhanga', 'Jamui', 'Samastipur', 'Janjgir', 'Kanker', 'Korba',
       'Koria', 'Narmada', 'Sirsa', 'Karwar(Uttar Kannad)',
       'Madikeri(Kodagu)', 'Rewa', 'Shivpuri', 'Sidhi', 'Parbhani',
       ' Jhunjhunu', 'Dudu', 'Jhalawar', 'Sanchore', 'Adilabad',
       'West Godavari', 'Kaithar', 'Kishanganj', 'Balodabazar', 'Bastar',
       'Jashpur', 'Kabirdham', 'Raigarh', 'Tumkur', 'Jhabua', 'Jalgaon',
       'Kalimpong', 'Krishna', 'Mungeli', 'Sukma', 'Surajpur', 'Bellary']

In [5]:
# Function to fetch weather data
def fetch_weather(city, api_key):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data for {city}: {response.status_code}")
        return None

# Fetch weather data for each city and print results
for city in cities:
    weather_data = fetch_weather(city, api_key)
    if weather_data:
        print(f"Weather in {city}: {weather_data['main']['temp']}°C, {weather_data['weather'][0]['description']}")

Error fetching data for North and Middle Andaman : 404
Error fetching data for Chittor: 404
Weather in Guntur: 27.04°C, heavy intensity rain
Weather in Visakhapatnam: 31.94°C, broken clouds
Weather in Barpeta: 29.9°C, clear sky
Weather in Cachar: 17.91°C, overcast clouds
Weather in Goalpara: 29.89°C, clear sky
Weather in Jorhat: 28.1°C, clear sky
Error fetching data for Kamrup: 404
Error fetching data for Karbi Anglong: 404
Weather in Kokrajhar: 29.26°C, clear sky
Weather in MORIGAON: 29.22°C, clear sky
Error fetching data for Sonitpur: 404
Weather in Araria: 31.1°C, few clouds
Weather in Bhojpur: 18.65°C, broken clouds
Weather in Madhubani: 30.09°C, broken clouds
Weather in Rohtas: 31.35°C, scattered clouds
Weather in Chandigarh: 32.51°C, clear sky
Weather in Bilaspur: 31.17°C, scattered clouds
Weather in Dhamtari: 31.37°C, overcast clouds
Weather in Durg: 31.1°C, scattered clouds
Weather in Kondagaon: 31.8°C, few clouds
Weather in Raipur: 31.08°C, haze
Weather in Rajnandgaon: 31.56°C

In [6]:
# Function to determine the current season based on Indian weather
def get_current_season():
    month = datetime.now().month
    if month in [12, 1, 2, 3]:  # Winter: December to early April
        return 'Winter'
    elif month in [4, 5, 6]:    # Summer or pre-monsoon: April to June (or July in NW India)
        return 'Summer'
    elif month in [7, 8, 9]:    # Monsoon or rainy: June to September
        return 'Monsoon'
    else:                       # Post-monsoon: October to December
        return 'Post-monsoon'


In [7]:
# Function to update weather DataFrame and save to CSV
def job():
    global weather_df
    for city in cities:
        weather_data = fetch_weather(city, api_key)
        if weather_data is not None:
            new_data = pd.DataFrame([{
                "timestamp": datetime.now(),
                "city": city,
                "temperature": weather_data['main']['temp'],
                "humidity": weather_data['main']['humidity'],
                "pressure": weather_data['main']['pressure'],
                "weather": weather_data['weather'][0]['description'],
                "wind_speed": weather_data['wind']['speed'],
                "season": get_current_season()
            }])
            weather_df = pd.concat([weather_df, new_data], ignore_index=True)

    # Save the DataFrame to CSV
    weather_df.to_csv('weather_data.csv', index=False)
    print("Updated weather DataFrame:")
    print(weather_df)

# For testing purposes, run the job function
job()
print("Final DataFrame:")
print(weather_df)

Error fetching data for North and Middle Andaman : 404
Error fetching data for Chittor: 404


C:\Users\pawan\AppData\Local\Temp\ipykernel_5132\2422838524.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weather_df = pd.concat([weather_df, new_data], ignore_index=True)


Error fetching data for Kamrup: 404
Error fetching data for Karbi Anglong: 404
Error fetching data for Sonitpur: 404
Error fetching data for North Goa: 404
Error fetching data for South Goa: 404
Error fetching data for Banaskanth: 404
Error fetching data for Devbhumi Dwarka: 404
Error fetching data for Gir Somnath: 404
Error fetching data for Kachchh: 404
Error fetching data for Mehsana: 404
Error fetching data for Panchmahals: 404
Error fetching data for Sabarkantha: 404
Error fetching data for Vadodara(Baroda): 404
Error fetching data for Jhajar: 404
Error fetching data for Mahendragarh-Narnaul: 404
Error fetching data for Mewat: 404
Error fetching data for Yamuna Nagar: 404
Error fetching data for Badgam: 404
Error fetching data for Mangalore(Dakshin Kannad): 404
Error fetching data for Kasargod: 404
Error fetching data for Kozhikode(Calicut): 404
Error fetching data for Palakad: 404
Error fetching data for Thirssur: 404
Error fetching data for Wayanad: 404
Error fetching data for A

In [8]:
# Function to schedule the job
schedule.every().day.at("08:00").do(job)  # Set to run daily at 08:00 AM

Every 1 day at 08:00:00 do job() (last run: [never], next run: 2024-10-15 08:00:00)

In [ ]:
# Run the scheduled jobs
while True:
    schedule.run_pending()
    time.sleep(1)  # Wait for one second before checking again